In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import gc
import torchvision.transforms as transforms

In [2]:
#device = torch.device("cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def PoissonGen(inp, rescale_fac=2.0):
    rand_inp = torch.rand_like(inp)
    return torch.mul(torch.le(rand_inp * rescale_fac, torch.abs(inp)).float(), torch.sign(inp))

# def spike_function(x):
#     x[x>0] = 1
#     x[x<=0] = 0
#     return x

def de_func(U,th):
    alpha = 0.3
    U = alpha*(1.0 - abs((U-th)/th))
    U[U<0]=0
    return U

def test(toy):
    test_loss = 0
    correct = 0
    toy = toy.cuda()
    for data, target in test_loader_cifar10:
        data = data.cuda()
        target = target.cuda()
        output = toy(data)
        test_loss +=F.cross_entropy(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader_cifar10.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader_cifar10.dataset),
        100. * correct / len(test_loader_cifar10.dataset)))

def quant(input, k):
    size = input.size()
    #mean = torch.mean(input.abs(), 1, keepdim=True)
    x = input
    #print(x)
    xmax = x.abs().max()
    num_bits=k
    v0 = 1
    v1 = 2
    v2 = -0.5
    y = k #2.**num_bits - 1.
    #print(y)
    x = x.add(v0).div(v1)
    #print(x)
    x = x.mul(y).round_()
    #print(x)
    x = x.div(y)
    #print(x)
    x = x.add(v2)
    #print(x)
    x = x.mul(v1)
    #print(x)
    input = x
    return input

def conv_weight_update(dH,X,pad):
    shap = dH.shape[1]
    dH = torch.sum(dH,0)
    dH = torch.unsqueeze(dH,1)
    dH = torch.repeat_interleave(dH,X.shape[1],0)
    X = X.repeat(1,shap,1,1)
    dw_conv = F.conv2d(X,dH,padding=pad,groups=X.shape[1])
    return dw_conv

def conv_dx_update(dH,W,pad):

    W = torch.transpose(W,0,1)
    W = torch.flip(W,[-1,-2]) # W = C*3*3
    dx_conv = F.conv2d(dH,W,padding=1)
    
    return dx_conv

In [4]:
class model(nn.Module):
    def __init__(self, time_step,leak):
        super(model, self).__init__()
        
        self.fc_1 = nn.Linear(28*28,256,bias=False)
        self.fc_2 = nn.Linear(256,256,bias=False)
        self.fc_out = nn.Linear(256,10,bias=False)
        
        self.lif1 = LIF(time_step,leak)
        self.lif2 = LIF(time_step,leak)
        self.time_step = time_step
        self.s_regs_inp = None
        
    def forward(self, inp):
        inp = inp.view(inp.shape[0],-1)
        size = inp.shape
        self.s_regs_inp = torch.zeros(self.time_step,*size, device=device)
        u_out = 0
        
        for t in range(self.time_step):
            
            spike_inp = PoissonGen(inp)
            self.s_regs_inp[t] += spike_inp 
            
            x = self.fc_1(spike_inp)
            #x = quant(x,2**4)
            x = self.lif1(x, t)
            x = self.fc_2(x)
            #x = quant(x,2**4)
            x = self.lif2(x, t)
            x = self.fc_out(x)
            u_out = u_out + x
        return u_out/self.time_step

In [5]:
class MLP(nn.Module):
    def __init__(self,time_step,leak):
        super(MLP, self).__init__()
        
        self.fc_1 = nn.Linear(28*28,512,bias=False)
        self.fc_out = nn.Linear(512,10,bias=False)
        self.lif1 = LIF(time_step,leak)
        self.time_step = time_step
        self.s_regs_inp = None
        
    def forward(self, inp):
#         print("size is:", (inp.view(inp.shape[0],1,28,28)).shape)
        inp = inp.view(inp.shape[0],-1)
        size = inp.shape
        
        self.s_regs_inp = torch.zeros(self.time_step,*size, device=device)
        u_out = 0
        
        for t in range(self.time_step):
            spike_inp = PoissonGen(inp)
            self.s_regs_inp[t] += spike_inp 
            x = self.fc_1(spike_inp)
            #x = quant(x,2**4)
            x = self.lif1(x, t)
            x = self.fc_out(x)
            u_out = u_out + x
        return u_out
        

In [6]:
class VGG_5(nn.Module):
    def __init__(self,time_step, leak):
        super(VGG_5, self).__init__()
        
        self.time_step = time_step
        self.s_regs_inp = None
        self.s_regs_conv = None
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False)
        self.conv_lif1 = LIF(time_step, leak)
        self.pool1 = nn.MaxPool2d(kernel_size=2,return_indices=True)
        self.pool1_ind = []
        self.unpool1 = nn.MaxUnpool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False)
        self.conv_lif2 = LIF(time_step, leak)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3, padding=1, bias=False)
        self.conv_lif3 = LIF(time_step, leak)
        self.pool2 = nn.MaxPool2d(kernel_size=2,return_indices=True)
        self.pool2_ind = []
        self.unpool2 = nn.MaxUnpool2d(kernel_size=2)

        self.fc1 = nn.Linear(128 * 8 * 8, 1024, bias=False)
        self.fc_lif1 = LIF(time_step,leak)
        self.fc_out = nn.Linear(1024, 10, bias=False)
        
    def forward(self, inp):

        size = inp.shape
        self.s_regs_inp = torch.zeros(self.time_step,*size, device=device)
        self.pool1_ind = []
        self.pool2_ind = []
        u_out = 0
        
        for t in range(self.time_step):
            spike_inp = PoissonGen(inp)
            self.s_regs_inp[t] += spike_inp 
            x = self.conv1(spike_inp)
            x = self.conv_lif1(x,t)
            x,indices = self.pool1(x)
            self.pool1_ind.append(indices)
            x = self.conv2(x)
            x = self.conv_lif2(x,t)
            x = self.conv3(x)
            x = self.conv_lif3(x,t)
            x,indices = self.pool2(x)
            x = x.view(x.shape[0],-1)
            
            if t == 0:
                self.s_regs_conv = torch.zeros(self.time_step,*x.shape, device=device)
            self.pool2_ind.append(indices)
            self.s_regs_conv[t] += x
            
            x = self.fc1(x)
            x = self.fc_lif1(x,t)
            x = self.fc_out(x)
            u_out = u_out + x
        return u_out

In [7]:
def bp_VGG5(vgg,leak,time_step,du_out,l_r,th):
   
    ## Update weight in FCs, time T
    du_fc1 = torch.matmul(du_out,vgg.fc_out.weight)*de_func(vgg.fc_lif1.u_regs[-1],th)
    vgg.fc_lif1.du_regs[-1] += du_fc1
    w_conv_1 = torch.matmul(torch.transpose(du_fc1,0,1),vgg.s_regs_conv[-1])
    vgg.fc1.weight.data -= l_r*w_conv_1   
    w_1_out = torch.matmul(torch.transpose(du_out,0,1),vgg.fc_lif1.s_regs[-1])
    vgg.fc_out.weight.data -= l_r*w_1_out
    
    ## Update du in pool2, time T
    dx_pool2 = torch.matmul(du_fc1,vgg.fc1.weight)
    dx_pool2 = dx_pool2.view(dx_pool2.shape[0],128,8,8)
    du_pool2 = vgg.unpool2(dx_pool2,vgg.pool2_ind[-1])
    
    ## Update du and dw in conv3, time T
    du_conv3 = du_pool2*de_func(toy.conv_lif3.u_regs[-1],th)
    vgg.conv_lif3.du_regs[-1] += du_conv3 
    dW_conv3 = conv_weight_update(du_pool2.type(torch.float),vgg.conv_lif2.s_regs[-1].type(torch.float),1)
    dW_conv3 = torch.sum(dW_conv3,0)
    dW_conv3 = dW_conv3.view(128,128,dW_conv3.shape[-1],dW_conv3.shape[-1])
    vgg.conv3.weight.data -=l_r*dW_conv3
    
    ## Update du and dw in conv2, time T
    
    du_conv2 = conv_dx_update(du_conv3,vgg.conv3.weight,'same')*de_func(toy.conv_lif2.u_regs[-1],th)
    vgg.conv_lif2.du_regs[-1] += du_conv2
    dW_conv2 = conv_weight_update(du_conv2.type(torch.float),F.max_pool2d(vgg.conv_lif1.s_regs[-1].type(torch.float),kernel_size=2),1)
    dW_conv2 = torch.sum(dW_conv2,0)
    dW_conv2 = dW_conv2.view(128,64,dW_conv2.shape[-1],dW_conv2.shape[-1])
    vgg.conv2.weight.data -=l_r*dW_conv2
    
    ## Update du in pool2, time t
    du_pool1 = vgg.unpool1(conv_dx_update(du_conv2,vgg.conv2.weight,'same'),vgg.pool1_ind[-1])
    
    du_conv1 = du_pool1*de_func(toy.conv_lif1.u_regs[-1],th)
    vgg.conv_lif1.du_regs[-1] += du_conv1
    dW_conv1 = conv_weight_update(du_conv1.type(torch.float),vgg.s_regs_inp[-1].type(torch.float),1)
    dW_conv1 = torch.sum(dW_conv1,0)
    dW_conv1 = dW_conv1.view(64,3,dW_conv1.shape[-1],dW_conv1.shape[-1])
    vgg.conv1.weight.data -=l_r*dW_conv1
    


    for t in range(time_step-2,-1,-1):
        
        ds_fc1 = torch.matmul(du_out,vgg.fc_out.weight)+vgg.fc_lif1.du_regs[t+1]*(-leak*vgg.fc_lif1.du_regs[t])
        du_fc1 = (ds_fc1)*de_func(vgg.fc_lif1.du_regs[t],th) + vgg.fc_lif1.du_regs[t+1]*leak*(1-vgg.fc_lif1.s_regs[t])
        vgg.fc_lif1.du_regs[t] += du_fc1
        w_conv_1 = torch.matmul(torch.transpose(du_fc1,0,1),vgg.s_regs_conv[t])
        vgg.fc1.weight.data -= l_r*w_conv_1   
        w_1_out = torch.matmul(torch.transpose(du_out,0,1),vgg.fc_lif1.s_regs[t])
        vgg.fc_out.weight.data -= l_r*w_1_out
        
        
        dx_pool2 = torch.matmul(du_fc1,vgg.fc1.weight)
        dx_pool2 = dx_pool2.view(dx_pool2.shape[0],128,8,8)
        du_pool2 = vgg.unpool2(dx_pool2,vgg.pool2_ind[t])
        ds_conv3 = du_pool2+vgg.conv_lif3.du_regs[t+1]*(-leak*vgg.conv_lif3.du_regs[t])
        du_conv3 = ds_conv3*de_func(toy.conv_lif3.u_regs[t],th) + vgg.conv_lif3.du_regs[t+1]*leak*(1-vgg.conv_lif3.s_regs[t])
        vgg.conv_lif3.du_regs[t] += du_conv3 
        dW_conv3 = conv_weight_update(du_pool2.type(torch.float),vgg.conv_lif2.s_regs[t].type(torch.float),1)
        dW_conv3 = torch.sum(dW_conv3,0)
        dW_conv3 = dW_conv3.view(128,128,dW_conv3.shape[-1],dW_conv3.shape[-1])
        vgg.conv3.weight.data -=l_r*dW_conv3
        
        ds_conv2 = conv_dx_update(du_conv3,vgg.conv3.weight,'same')+vgg.conv_lif2.du_regs[t+1]*(-leak*vgg.conv_lif2.du_regs[t])
        du_conv2 = ds_conv2*de_func(toy.conv_lif2.u_regs[t],th) + vgg.conv_lif2.du_regs[t+1]*leak*(1-vgg.conv_lif2.s_regs[t])
        vgg.conv_lif2.du_regs[t] += du_conv2 
        dW_conv2 = conv_weight_update(du_conv2.type(torch.float),F.max_pool2d(vgg.conv_lif1.s_regs[t].type(torch.float),kernel_size=2),1)
        dW_conv2 = torch.sum(dW_conv2,0)
        dW_conv2 = dW_conv2.view(128,64,dW_conv2.shape[-1],dW_conv2.shape[-1])
        vgg.conv2.weight.data -=l_r*dW_conv2
        
        
        du_pool1 = vgg.unpool1(conv_dx_update(du_conv2,vgg.conv2.weight,'same'),vgg.pool1_ind[t])
        ds_conv1 = du_pool1 + vgg.conv_lif1.du_regs[t+1]*(-leak*vgg.conv_lif1.du_regs[t])
        du_conv1 = ds_conv1*de_func(toy.conv_lif1.u_regs[t],th) + vgg.conv_lif1.du_regs[t+1]*leak*(1-vgg.conv_lif1.s_regs[t])
        vgg.conv_lif1.du_regs[t] += du_conv1
        dW_conv1 = conv_weight_update(du_conv1.type(torch.float),vgg.s_regs_inp[t].type(torch.float),1)
        dW_conv1 = torch.sum(dW_conv1,0)
        dW_conv1 = dW_conv1.view(64,3,dW_conv1.shape[-1],dW_conv1.shape[-1])
        vgg.conv1.weight.data -=l_r*dW_conv1
    
    return 0

In [8]:
#     du_pool2 = torch.sum(du_pool2,0)
#     du_pool2 = torch.unsqueeze(du_pool2,1)
    
    ## Update du in conv3, time T
#     d_conv3 = nn.Conv2d(128, 128, stride=1, kernel_size=f, padding=f-1, bias=False)
    
    ## Update weight in Conv3, time T
#     f = du_pool2.shape[-1]
#     d_conv3 = nn.Conv2d(128, 128, stride=1, padding=1, kernel_size=f, bias=False)
#     d_conv3.weight.data = du_pool2.type(torch.float)
#     dW_conv3 = d_conv3(vgg.conv_lif2.s_regs[-1].type(torch.float))
#     dW_conv3 = torch.sum(dW_conv3,0)
#     dW_conv3 = torch.unsqueeze(dW_conv3,1)
#     vgg.conv3.weight.data -= l_r*dW_conv3

In [9]:
class VGG_1(nn.Module):
    def __init__(self,time_step,leak):
        super(VGG_1, self).__init__()
        
        self.time_step = time_step
        self.s_regs_inp = None
        self.s_regs_conv = None
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1, bias=False)
        
#         self.deconv1 = nn.Conv2d()
        self.lif_conv1 = LIF(time_step,leak)
        self.pool1 = nn.MaxPool2d(kernel_size=2,return_indices=True)
        self.pool1_ind = []
        self.unpool1 = nn.MaxUnpool2d(kernel_size=2)

        self.fc1 = nn.Linear(16 * 14 * 14, 512, bias=False)
        self.lif_fc1 = LIF(time_step,leak)
        self.fc_out = nn.Linear(512, 10, bias=False)
        
    def forward(self, inp):

        size = inp.shape
        self.s_regs_inp = torch.zeros(self.time_step,*size, device=device)
        self.pool1_ind = []

        u_out = 0
        for t in range(self.time_step):
            spike_inp = PoissonGen(inp)
            self.s_regs_inp[t] += spike_inp
            x = self.conv1(spike_inp)
            x = self.lif_conv1(x,t)
            x, indices = self.pool1(x)
            x= x.view(x.shape[0],-1)
            
            if t == 0:
                self.s_regs_conv = torch.zeros(self.time_step,*x.shape, device=device)
            self.pool1_ind.append(indices)
            self.s_regs_conv[t] += x
            
            x = self.fc1(x)
            x = self.lif_fc1(x,t)
            
            x = self.fc_out(x)
            u_out = u_out + x

        return u_out

In [10]:
def bp_VGG1(vgg,leak,time_step,du_out,l_r,th):
   
    ## First fc
    du_fc1 = torch.matmul(du_out,vgg.fc_out.weight)*de_func(vgg.lif_fc1.u_regs[-1],th)
    vgg.lif_fc1.du_regs[-1] += du_fc1
       
    ## Update weight
    w_conv_1 = torch.matmul(torch.transpose(du_fc1,0,1),vgg.s_regs_conv[-1])
    vgg.fc1.weight.data -= l_r*w_conv_1
     
    w_1_out = torch.matmul(torch.transpose(du_out,0,1),vgg.lif_fc1.s_regs[-1])
    vgg.fc_out.weight.data -= l_r*w_1_out
    
    dx_pool1 = torch.matmul(du_fc1,vgg.fc1.weight)
    dx_pool1 = dx_pool1.view(dx_pool1.shape[0],16,14,14)
    du_pool1 = vgg.unpool1(dx_pool1,vgg.pool1_ind[-1])
    du_pool1 = torch.sum(du_pool1,0)
    du_pool1 = torch.unsqueeze(du_pool1,1)
    f = du_pool1.shape[-1]
    d_conv1_w = nn.Conv2d(1, 16, stride=1, padding=1,kernel_size=f, bias=False)
    d_conv1_w.weight.data = du_pool1.type(torch.float)
    dW = d_conv1_w(vgg.s_regs_inp[-1].type(torch.float))
    dW = torch.sum(dW,0)
    dW = torch.unsqueeze(dW,1)

    vgg.conv1.weight.data -= l_r*dW
    
    for t in range(time_step-2,-1,-1):
        
        ds_fc1 = torch.matmul(du_out,vgg.fc_out.weight)+vgg.lif_fc1.du_regs[t+1]*(-leak*vgg.lif_fc1.du_regs[t])
        du_fc1 = (ds_fc1)*de_func(vgg.lif_fc1.du_regs[t],th) + vgg.lif_fc1.du_regs[t+1]*leak*(1-vgg.lif_fc1.s_regs[t])
        vgg.lif_fc1.du_regs[t] += du_fc1
        
        w_conv_1 = torch.matmul(torch.transpose(du_fc1,0,1),vgg.s_regs_conv[t])
        vgg.fc1.weight.data -= l_r*w_conv_1
        
        
        dx_pool1 = torch.matmul(du_fc1,vgg.fc1.weight)
        dx_pool1 = dx_pool1.view(dx_pool1.shape[0],16,14,14)
        du_pool1 = vgg.unpool1(dx_pool1,vgg.pool1_ind[t])
        du_pool1 = torch.sum(du_pool1,0)
        du_pool1 = torch.unsqueeze(du_pool1,1)
        f = du_pool1.shape[-1]
        d_conv1_w = nn.Conv2d(1, 16, stride=1, padding=1,kernel_size=f, bias=False)
        d_conv1_w.weight.data = du_pool1.type(torch.float)
        dW = d_conv1_w(vgg.s_regs_inp[t].type(torch.float))
        dW = torch.sum(dW,0)
        dW = torch.unsqueeze(dW,1)

        vgg.conv1.weight.data -= l_r*dW
    
    
    return 0
    
    

In [11]:
class LIF(nn.Module):
    def __init__(self, time_step,leak):
        super(LIF, self).__init__()
        
        self.u_regs = None
        self.du_regs = None
        self.s_regs = None
        self.leak = leak
        self.time_step = time_step
        self.thresh = 0.7
        
    def forward(self,inp,t):
        
#         print("memory before clear",torch.cuda.memory_allocated())
        if t == 0:
            size = inp.shape
            self.u_regs = torch.zeros(self.time_step,*size, device=device)
            self.du_regs = torch.zeros(self.time_step,*size, device=device)
#             err = torch.normal(0, 0.1,(1,1)).cuda()
#             inp = inp + err
#             self.u_regs[0] = quant(inp,2**4)
            self.u_regs[0] = inp
            self.s_regs = torch.zeros(self.time_step,*size, device=device)

            spike = inp.gt(self.thresh).float()

            self.s_regs[0] = spike
            
        else:
#             err = torch.normal(0, 0.1,(1,1))
#             inp = inp + err
#             self.u_regs[t] = quant(self.leak * self.u_regs[t-1] * (1 - self.s_regs[t-1]) + (1-self.leak)*inp, 2**4)
            self.u_regs[t] = self.leak*self.u_regs[t-1]*(1-self.s_regs[t-1]) + inp

            spike = self.u_regs[t].gt(self.thresh).float()

            self.s_regs[t] = spike
            
#         print("memory after clear",torch.cuda.memory_allocated())
#         torch.cuda.empty_cache()
#         gc.collect()
        return spike


In [12]:
### Back propagation for MLP
def bp_MLP(toy,leak,time_step,du_out,s_regs_inp,l_r,th):
    
    ## First fc
    du_fc1 = torch.matmul(du_out,toy.fc_out.weight)*de_func(toy.lif1.u_regs[-1],th)
    toy.lif1.du_regs[-1] += du_fc1

    ## Update weight
    w_inp_1 = torch.matmul(torch.transpose(du_fc1,0,1),s_regs_inp[-1])
#     toy.fc_1.weight.data -= l_r*quant(w_inp_1,2**4)
    toy.fc_1.weight.data -= l_r*w_inp_1
    
    w_1_out = torch.matmul(torch.transpose(du_out,0,1),toy.lif1.s_regs[-1])
#     toy.fc_out.weight.data -= l_r*quant(w_1_out,2**4)
    toy.fc_out.weight.data -= l_r*w_1_out

    for t in range(time_step-2,-1,-1):
        
        ## First fc
        ds_fc1 = torch.matmul(du_out,toy.fc_out.weight)+toy.lif1.du_regs[t+1]*(-leak*toy.lif1.u_regs[t])
        du_fc1 = (ds_fc1)*de_func(toy.lif1.u_regs[t],th) + toy.lif1.du_regs[t+1]*leak*(1-toy.lif1.s_regs[t])
        toy.lif1.du_regs[t] += du_fc1

        ## Update weight
        w_inp_1 = torch.matmul(torch.transpose(du_fc1,0,1),s_regs_inp[t])
#         toy.fc_1.weight.data -= l_r*quant(w_inp_1,2**4)
#         print("du_size",du_fc1.shape)
#         print("s_size",s_regs_inp[t].shape)
#         print("dweight shape",w_inp_1.shape)
        toy.fc_1.weight.data -= l_r*w_inp_1

        w_1_out = torch.matmul(torch.transpose(du_out,0,1),toy.lif1.s_regs[t])
#         toy.fc_out.weight.data -= l_r*quant(w_1_out,2**4)
        toy.fc_out.weight.data -= l_r*w_1_out


In [13]:
### Back propagation
def bp(toy,leak,time_step,du_out,s_regs_inp,l_r,th):
    
    ## Second fc    
    du_fc2 = torch.matmul(du_out,toy.fc_out.weight)*de_func(toy.lif2.u_regs[-1],th)    
    toy.lif2.du_regs[-1] = toy.lif2.du_regs[-1] + du_fc2
    
    ## First fc
    du_fc1 = torch.matmul(du_fc2,toy.fc_2.weight)*de_func(toy.lif1.u_regs[-1],th)
    toy.lif1.du_regs[-1] += du_fc1

    
    ## Update weight
    w_inp_1 = torch.matmul(torch.transpose(du_fc1,0,1),s_regs_inp[-1])
    toy.fc_1.weight.data -= l_r*quant(w_inp_1,2**4)
    #toy.fc_1.weight.data -= l_r*w_inp_1

    w_1_2 = torch.matmul(torch.transpose(du_fc2,0,1),toy.lif1.s_regs[-1])
    toy.fc_2.weight.data -= l_r*quant(w_1_2,2**4)
    #toy.fc_2.weight.data -= l_r*w_1_2

    w_2_out = torch.matmul(torch.transpose(du_out,0,1),toy.lif2.s_regs[-1])
    toy.fc_out.weight.data -= l_r*quant(w_2_out,2**4)
    #toy.fc_out.weight.data -= l_r*w_2_out

    for t in range(time_step-2,-1,-1):

        ds_fc2 = torch.matmul(du_out,toy.fc_out.weight)+toy.lif2.du_regs[t+1]*(-leak*toy.lif2.u_regs[t])
        du_fc2 = (ds_fc2)*de_func(toy.lif2.u_regs[t],th) + toy.lif2.du_regs[t+1]*leak*(1-toy.lif2.s_regs[t])
        toy.lif2.du_regs[t] += du_fc2
        
        ## First fc
        ds_fc1 = torch.matmul(du_fc2,toy.fc_2.weight)+toy.lif1.du_regs[t+1]*(-leak*toy.lif1.u_regs[t])
        du_fc1 = (ds_fc1)*de_func(toy.lif1.u_regs[t],th) + toy.lif1.du_regs[t+1]*leak*(1-toy.lif1.s_regs[t])
        toy.lif1.du_regs[t] += du_fc1

        ## Update weight
        w_inp_1 = torch.matmul(torch.transpose(du_fc1,0,1),s_regs_inp[t])
        toy.fc_1.weight.data -= l_r*quant(w_inp_1,2**4)
        
        #toy.fc_1.weight.data -= l_r*w_inp_1

        w_1_2 = torch.matmul(torch.transpose(du_fc2,0,1),toy.lif1.s_regs[t])
        toy.fc_2.weight.data -= l_r*quant(w_1_2,2**4)
        #toy.fc_2.weight.data -= l_r*w_1_2

        w_2_out = torch.matmul(torch.transpose(du_out,0,1),toy.lif2.s_regs[t])
        toy.fc_out.weight.data -= l_r*quant(w_2_out,2**4)
        #toy.fc_out.weight.data -= l_r*w_2_out


In [14]:
import torch
import torchvision

batch_size_train = 64
batch_size_test = 100

train_loader_mnist = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./mnist', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader_mnist = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./mnist', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

C:\Users\Ruokai\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [15]:
import urllib.request
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
response = urllib.request.urlopen('https://www.python.org')
print(response.read().decode('utf-8'))

<!doctype html>
<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->
<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->
<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->
<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->

<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">
    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jqueryui/1.12.1/jquery-ui.min.js">

    <meta name="application-name" content="Python.org">
    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">
    <meta name="apple-mobile-web-app-title" content="Python.org">
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">

    <meta name="

In [16]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True,
                                        download=True, transform=transform)
train_loader_cifar10 = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4,pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False,
                                       download=True, transform=transform)
test_loader_cifar10 = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=4,pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [17]:
test_losses = []
log_interval = 10

In [18]:
time_step = 15
leak = 0.99
toy = VGG_5(time_step,leak).cuda()
# vgg = VGG_5(time_step)
# vgg =vgg.cuda()
# print("weight",toy.fc_1.weight)
# torch.nn.init.normal_(toy.fc_1.weight, mean=0.0, std=0.1)
# toy.fc_1.weight.data = quant(toy.fc_1.weight,2**4)
# torch.nn.init.normal_(toy.fc_2.weight, mean=0.0, std=0.1)
# toy.fc_2.weight.data = quant(toy.fc_2.weight,2**4)
# torch.nn.init.normal_(toy.fc_out.weight, mean=0.0, std=0.1)
# toy.fc_out.weight.data = quant(toy.fc_out.weight,2**4)
# print("quantized weight",toy.fc_1.weight)
lr = 0.0008
loss = nn.CrossEntropyLoss()

test(toy)
with torch.no_grad():
    for epoch in range(12):
        for batch_idx, (data, target) in enumerate(train_loader_cifar10):
            data = data.cuda()
            target = target.cuda()
            out = toy(data)
#             print("memory after fwd",torch.cuda.memory_allocated()/10000000)

            err = loss(out,target)

            exp = torch.exp(out)
            exp_sum = torch.sum(torch.exp(out),1, keepdim=True)   
            target = F.one_hot(target, num_classes=10)
            #L = -1*torch.sum((target*torch.log((exp/exp_sum))),1, keepdim=True)
            du_out = exp/exp_sum
            du_out = (du_out - target)/batch_size_train



    #         vgg_out = vgg(data)
    #         exp_vgg = torch.exp(vgg_out)
    #         exp_sum_vgg = torch.sum(torch.exp(vgg_out),1, keepdim=True)
    #         du_out_vgg = exp_vgg/exp_sum_vgg
    #         du_out_vgg = du_out_vgg - target
    #         print(du_out_vgg)



            bp_VGG5(toy,leak,time_step,du_out,lr,0.7)
#             print("memory after bp",torch.cuda.memory_allocated()/10000000)

    #         bp_MLP(toy,leak,time_step,du_out,toy.s_regs_inp,lr,toy.lif1.thresh)



            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader_cifar10.dataset),
                100. * batch_idx / len(train_loader_cifar10), err.item()))

#             del toy.lif_conv1.s_regs
#             del toy.lif_conv1.u_regs
#             del toy.lif_conv1.du_regs
#             del toy.lif_fc1.du_regs
#             del toy.lif_fc1.u_regs
#             del toy.lif_fc1.s_regs
#             del toy.s_regs_conv
#             del toy.s_regs_inp
#             del data
#             del target
#             torch.cuda.empty_cache()
            
#             gc.collect()
#             print("memory after clear",torch.cuda.memory_allocated()/10000000)

        test(toy)


C:\Users\Ruokai\anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:693: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch._C._nn.max_pool2d_with_indices(input, kernel_size, stride, padding, dilation, ceil_mode)
C:\Users\Ruokai\anaconda3\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.4026, Accuracy: 1103/10000 (11%)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.512688
Train Epoch: 0 [320/50000 (1%)]	Loss: 2.506984
Train Epoch: 0 [640/50000 (1%)]	Loss: 2.330268
Train Epoch: 0 [960/50000 (2%)]	Loss: 2.355562
Train Epoch: 0 [1280/50000 (3%)]	Loss: 2.219333
Train Epoch: 0 [1600/50000 (3%)]	Loss: 2.286567
Train Epoch: 0 [1920/50000 (4%)]	Loss: 2.224273
Train Epoch: 0 [2240/50000 (4%)]	Loss: 2.263476
Train Epoch: 0 [2560/50000 (5%)]	Loss: 2.251234
Train Epoch: 0 [2880/50000 (6%)]	Loss: 2.239772
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.343350
Train Epoch: 0 [3520/50000 (7%)]	Loss: 2.286649
Train Epoch: 0 [3840/50000 (8%)]	Loss: 2.286069
Train Epoch: 0 [4160/50000 (8%)]	Loss: 2.237209
Train Epoch: 0 [4480/50000 (9%)]	Loss: 2.208657
Train Epoch: 0 [4800/50000 (10%)]	Loss: 2.220046
Train Epoch: 0 [5120/50000 (10%)]	Loss: 2.363691
Train Epoch: 0 [5440/50000 (11%)]	Loss: 2.225754
Train Epoch: 0 [5760/50000 (12%)]	Loss: 2.212921
Train Epoch: 0 [6080/50000 (12%)

Train Epoch: 1 [1920/50000 (4%)]	Loss: 1.926830
Train Epoch: 1 [2240/50000 (4%)]	Loss: 1.946218
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.088883
Train Epoch: 1 [2880/50000 (6%)]	Loss: 1.863091
Train Epoch: 1 [3200/50000 (6%)]	Loss: 1.969249
Train Epoch: 1 [3520/50000 (7%)]	Loss: 2.156210
Train Epoch: 1 [3840/50000 (8%)]	Loss: 1.845936
Train Epoch: 1 [4160/50000 (8%)]	Loss: 1.947160
Train Epoch: 1 [4480/50000 (9%)]	Loss: 1.936581
Train Epoch: 1 [4800/50000 (10%)]	Loss: 1.805771
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.272000
Train Epoch: 1 [5440/50000 (11%)]	Loss: 1.967933
Train Epoch: 1 [5760/50000 (12%)]	Loss: 1.975758
Train Epoch: 1 [6080/50000 (12%)]	Loss: 1.979412
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.790987
Train Epoch: 1 [6720/50000 (13%)]	Loss: 2.087777
Train Epoch: 1 [7040/50000 (14%)]	Loss: 2.094913
Train Epoch: 1 [7360/50000 (15%)]	Loss: 1.924516
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.794707
Train Epoch: 1 [8000/50000 (16%)]	Loss: 2.008805
Train Epoch: 1 [8320/50000 (1

Train Epoch: 2 [4480/50000 (9%)]	Loss: 1.753031
Train Epoch: 2 [4800/50000 (10%)]	Loss: 1.848121
Train Epoch: 2 [5120/50000 (10%)]	Loss: 1.752219
Train Epoch: 2 [5440/50000 (11%)]	Loss: 1.927463
Train Epoch: 2 [5760/50000 (12%)]	Loss: 1.933768
Train Epoch: 2 [6080/50000 (12%)]	Loss: 1.705509
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.721266
Train Epoch: 2 [6720/50000 (13%)]	Loss: 2.169960
Train Epoch: 2 [7040/50000 (14%)]	Loss: 1.864570
Train Epoch: 2 [7360/50000 (15%)]	Loss: 1.762724
Train Epoch: 2 [7680/50000 (15%)]	Loss: 1.880111
Train Epoch: 2 [8000/50000 (16%)]	Loss: 1.875964
Train Epoch: 2 [8320/50000 (17%)]	Loss: 1.886298
Train Epoch: 2 [8640/50000 (17%)]	Loss: 1.742773
Train Epoch: 2 [8960/50000 (18%)]	Loss: 1.843654
Train Epoch: 2 [9280/50000 (19%)]	Loss: 1.651989
Train Epoch: 2 [9600/50000 (19%)]	Loss: 1.833954
Train Epoch: 2 [9920/50000 (20%)]	Loss: 1.937297
Train Epoch: 2 [10240/50000 (20%)]	Loss: 1.846593
Train Epoch: 2 [10560/50000 (21%)]	Loss: 1.844962
Train Epoch: 2 [108

Train Epoch: 3 [6720/50000 (13%)]	Loss: 1.760059
Train Epoch: 3 [7040/50000 (14%)]	Loss: 2.159623
Train Epoch: 3 [7360/50000 (15%)]	Loss: 1.800922
Train Epoch: 3 [7680/50000 (15%)]	Loss: 2.089542
Train Epoch: 3 [8000/50000 (16%)]	Loss: 1.794388
Train Epoch: 3 [8320/50000 (17%)]	Loss: 1.855988
Train Epoch: 3 [8640/50000 (17%)]	Loss: 1.733113
Train Epoch: 3 [8960/50000 (18%)]	Loss: 1.850222
Train Epoch: 3 [9280/50000 (19%)]	Loss: 1.929007
Train Epoch: 3 [9600/50000 (19%)]	Loss: 1.985541
Train Epoch: 3 [9920/50000 (20%)]	Loss: 1.750042
Train Epoch: 3 [10240/50000 (20%)]	Loss: 1.723560
Train Epoch: 3 [10560/50000 (21%)]	Loss: 1.711757
Train Epoch: 3 [10880/50000 (22%)]	Loss: 1.842379
Train Epoch: 3 [11200/50000 (22%)]	Loss: 1.900071
Train Epoch: 3 [11520/50000 (23%)]	Loss: 1.992029
Train Epoch: 3 [11840/50000 (24%)]	Loss: 1.835711
Train Epoch: 3 [12160/50000 (24%)]	Loss: 1.912574
Train Epoch: 3 [12480/50000 (25%)]	Loss: 1.952109
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1.931376
Train Epoch

Train Epoch: 4 [8960/50000 (18%)]	Loss: 1.814336
Train Epoch: 4 [9280/50000 (19%)]	Loss: 2.041853
Train Epoch: 4 [9600/50000 (19%)]	Loss: 1.848240
Train Epoch: 4 [9920/50000 (20%)]	Loss: 1.836654
Train Epoch: 4 [10240/50000 (20%)]	Loss: 1.807689
Train Epoch: 4 [10560/50000 (21%)]	Loss: 1.816464
Train Epoch: 4 [10880/50000 (22%)]	Loss: 1.751378
Train Epoch: 4 [11200/50000 (22%)]	Loss: 1.872334
Train Epoch: 4 [11520/50000 (23%)]	Loss: 1.913375
Train Epoch: 4 [11840/50000 (24%)]	Loss: 1.934407
Train Epoch: 4 [12160/50000 (24%)]	Loss: 1.852543
Train Epoch: 4 [12480/50000 (25%)]	Loss: 1.850769
Train Epoch: 4 [12800/50000 (26%)]	Loss: 1.753243
Train Epoch: 4 [13120/50000 (26%)]	Loss: 1.816702
Train Epoch: 4 [13440/50000 (27%)]	Loss: 1.853896
Train Epoch: 4 [13760/50000 (28%)]	Loss: 1.775591
Train Epoch: 4 [14080/50000 (28%)]	Loss: 1.937145
Train Epoch: 4 [14400/50000 (29%)]	Loss: 1.844877
Train Epoch: 4 [14720/50000 (29%)]	Loss: 1.749787
Train Epoch: 4 [15040/50000 (30%)]	Loss: 1.769358
Trai

Train Epoch: 5 [11200/50000 (22%)]	Loss: 1.708094
Train Epoch: 5 [11520/50000 (23%)]	Loss: 1.953123
Train Epoch: 5 [11840/50000 (24%)]	Loss: 1.651391
Train Epoch: 5 [12160/50000 (24%)]	Loss: 2.023808
Train Epoch: 5 [12480/50000 (25%)]	Loss: 1.853244
Train Epoch: 5 [12800/50000 (26%)]	Loss: 1.865480
Train Epoch: 5 [13120/50000 (26%)]	Loss: 1.873628
Train Epoch: 5 [13440/50000 (27%)]	Loss: 1.915554
Train Epoch: 5 [13760/50000 (28%)]	Loss: 1.943027
Train Epoch: 5 [14080/50000 (28%)]	Loss: 1.772779
Train Epoch: 5 [14400/50000 (29%)]	Loss: 2.004988
Train Epoch: 5 [14720/50000 (29%)]	Loss: 2.089958
Train Epoch: 5 [15040/50000 (30%)]	Loss: 1.875651
Train Epoch: 5 [15360/50000 (31%)]	Loss: 1.699066
Train Epoch: 5 [15680/50000 (31%)]	Loss: 1.739483
Train Epoch: 5 [16000/50000 (32%)]	Loss: 2.055350
Train Epoch: 5 [16320/50000 (33%)]	Loss: 2.003583
Train Epoch: 5 [16640/50000 (33%)]	Loss: 1.926216
Train Epoch: 5 [16960/50000 (34%)]	Loss: 1.949093
Train Epoch: 5 [17280/50000 (35%)]	Loss: 1.808292


Train Epoch: 6 [13440/50000 (27%)]	Loss: 1.684497
Train Epoch: 6 [13760/50000 (28%)]	Loss: 1.801134
Train Epoch: 6 [14080/50000 (28%)]	Loss: 1.694059
Train Epoch: 6 [14400/50000 (29%)]	Loss: 1.731117
Train Epoch: 6 [14720/50000 (29%)]	Loss: 1.724581
Train Epoch: 6 [15040/50000 (30%)]	Loss: 2.092232
Train Epoch: 6 [15360/50000 (31%)]	Loss: 2.014477
Train Epoch: 6 [15680/50000 (31%)]	Loss: 1.743608
Train Epoch: 6 [16000/50000 (32%)]	Loss: 1.886157
Train Epoch: 6 [16320/50000 (33%)]	Loss: 1.629804
Train Epoch: 6 [16640/50000 (33%)]	Loss: 1.726549
Train Epoch: 6 [16960/50000 (34%)]	Loss: 1.661667
Train Epoch: 6 [17280/50000 (35%)]	Loss: 2.036995
Train Epoch: 6 [17600/50000 (35%)]	Loss: 1.659845
Train Epoch: 6 [17920/50000 (36%)]	Loss: 1.864445
Train Epoch: 6 [18240/50000 (36%)]	Loss: 1.825259
Train Epoch: 6 [18560/50000 (37%)]	Loss: 1.877351
Train Epoch: 6 [18880/50000 (38%)]	Loss: 1.903811
Train Epoch: 6 [19200/50000 (38%)]	Loss: 1.957149
Train Epoch: 6 [19520/50000 (39%)]	Loss: 1.880522


KeyboardInterrupt: 